In [1]:
import json
from io import open

# Path to the .jsonl file
file_path = "2020_acl_diplomacy/data/train.jsonl"
diplomacy_data = {}

with open(file_path, 'r') as file:
    i = 0
    for line in file:
        # Parse the line as JSON
        diplomacy_data[i] = json.loads(line)
        i+=1




In [7]:
# print(len(diplomacy_data))
for key, value in diplomacy_data[27].items():
    print("Key: ", key)
messages_1= diplomacy_data[26]["messages"]
for message in messages_1:
    print(message)


Key:  messages
Key:  sender_labels
Key:  receiver_labels
Key:  speakers
Key:  receivers
Key:  absolute_message_index
Key:  relative_message_index
Key:  seasons
Key:  years
Key:  game_score
Key:  game_score_delta
Key:  players
Key:  game_id
Gahhh I’m Italy fuck my life
Non aggression pact for these first two turns?  Seems to me we both have other things to work about.  Lmk what you’re thinking.
Sounds good with me
Cool.  So no moves towards each other?
Yeah that's right. I'm not moving into Piedmont
Rgr- neither will I
Sounds good with me
You can take Tunis and we can demilitarize the Western Med area while we deal with our more pressing concerns
For sure man.  I think we got other stuff going on lol
Was that bounce with France on purpose?
*germany my b
I thought he;d move into Ruhr like normal, but it wasnt too bad for me
Should’ve stabbed austria
Fu kkk I’m so screwed
If you send your armies west uou don’t have to be
I’ve no desire for an internecine conflict
Cool cool I’m tracking.  